<a href="https://colab.research.google.com/github/MeGaNeKoS/Google_Colab/blob/master/GDrive_Video_converting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GDrive Downloader 

In [ ]:
#@title Mount Google Drive
#@markdown To stream files we need to mount Google Drive.

from google.colab import drive
drive.mount('/content/drive')

# FFMPEG video converter

In [ ]:
#@title Install requirements
#@markdown To stream files we need to run ffmpeg.
!apt install ffmpeg

### Convert video
Convert the video format using ffmpeg

In [ ]:
#@title Converting
#@markdown fill the parameter before converting. also check your notebook setting before using GPU

import os
from IPython.display import clear_output
import threading
import time

PATH_ORI = "/content/drive/Shared drives/Anime/judas"#@param {type:"string"}

#@markdown Video Format
IN_FORMAT = "mkv" #@param {type:"string"}
OUT_FORMAT = "mp4" #@param {type:"string"}
#@markdown Video codec
METHODE = "h.265" #@param ["copy", "copy (-strict experimental)", "h.265"]
#@markdown Delete the source after convertion?
DELETE_SC = True #@param {type:"boolean"}
#@markdown Include the sub folder?
SUB_FOLDER = True #@param {type:"boolean"}
#@markdown Use GPU?
GPU = True #@param {type:"boolean"}
#CLEAN_OUTPUT = False #@param {type:"boolean"}

BEGIN = time.time() #Set start time

def sizeof_fmt(num, suffix='B'):
    for unit in ['','K','M','G','T','P','E','Z']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

def duration(num,msg):
    if int((num-BEGIN)//3600)>0:
        return f"{msg} : {int((num-BEGIN)//3600)} hour : {int((num-BEGIN)//60)%60} min : {int((num-BEGIN)%60)} sec"
    elif int((num-BEGIN)//60)>0:
        return f"{msg} : {int((num-BEGIN)//60)} min : {int((num-BEGIN)%60)} sec"
    else:
        return f"{msg} : {int((num-BEGIN)%60)} sec"

def convert(src, param, out):
  global waiting
  !ffmpeg -hide_banner -loglevel panic -y -i "$src" $param "$out"
  waiting = False

files = []
fail = []
reenc = []

ext = '.' + IN_FORMAT

# r=root, d=directories, f = files
if SUB_FOLDER:
    for r, d, f in os.walk(PATH_ORI, topdown=True):
        for file in f:
            if ext in file:
                files.append(os.path.join(r, file))
elif not SUB_FOLDER:
    for name in os.listdir(PATH_ORI):
        if ext in name:
            files.append(name)

if IN_FORMAT == OUT_FORMAT:
    PATH_ALT = PATH_ORI + "/converted"
    DELETE_SC = False
    if not os.path.isdir(PATH_ALT):
        os.makedirs(PATH_ALT)
    if SUB_FOLDER:
        for r, d, f in os.walk(PATH_ORI, topdown=True):
            child = r.replace(PATH_ORI, PATH_ALT)
            if not os.path.isdir(child):
                os.mkdir(child)

files.sort()
MAX = len(files)
for idx, f in enumerate(files):
    if ext in f :
        if SUB_FOLDER == False:
            source = PATH_ORI + "/" + f
            output = PATH_ORI + "/" + f.replace("." + IN_FORMAT, "." + OUT_FORMAT)
        else :
            source = f
            output = f.replace("." + IN_FORMAT, "." + OUT_FORMAT)
        
        if IN_FORMAT == OUT_FORMAT:
            output = output.replace(PATH_ORI, "")
            output = PATH_ALT + output

        temp = f"{str(f).split('/')[-1:][0]}\nConverted {idx} of {MAX}\n"
        tempt = f"{duration(time.time(),'Time Elapsed')}"
        print (temp + tempt,end='')
        title = len(temp)
        print ("\b"*len(tempt),end='')

        if METHODE == "copy" :
            #!ffmpeg -hide_banner -loglevel panic -y -i "$source" -map 0:a -map 0:v:0 -c:v copy -c:a aac -q:a 1 -map_chapters 0 -map_metadata 0 "$output"
            t1 = threading.Thread(target=convert, args=(source, '-map 0:a -map 0:v:0 -c:v copy -c:a aac -q:a 1 -map_chapters 0 -map_metadata 0', Vx265), daemon = True)
            waiting = True
            t1.start()
            while waiting:
              tempt = f"{duration(time.time(),'Time Elapsed')}" #Duration
              print(f'\r{tempt}', end='')
              time.sleep(1)
              
        elif METHODE == "copy (-strict experimental)" :
            #!ffmpeg -hide_banner -loglevel panic -y -i "$source" -map 0:a -map 0:v:0 -c:v copy -c:a copy -map_chapters 0 -map_metadata 0 -strict experimental "$output"
            t1 = threading.Thread(target=convert, args=(source, '-map 0:a -map 0:v:0 -c:v copy -c:a aac -q:a 1 -map_chapters 0 -map_metadata 0 -strict experimental', Vx265), daemon = True)
            waiting = True
            t1.start()
            while waiting:
              tempt = f"{duration(time.time(),'Time Elapsed')}" #Duration
              print(f'\r{tempt}', end='')
              time.sleep(1)

        elif METHODE == "h.265":
            GPU_type = ""
            if GPU:
                ##Set up Variable
                Vx265 = f"Vx265.{OUT_FORMAT}"
                Vcopy = f"Vcopy.{OUT_FORMAT}"
                Aud = f"Audio.{OUT_FORMAT}"
                flag = True

                temp = f"Converting the video using h.265\n" #Status
               
                print (temp,end='')
                
                #!ffmpeg -hide_banner -loglevel panic $GPU_type -y -i "$source" -map 0:v:0 -c:v hevc_nvenc -preset hq -profile:v main10 -tier high -rc:v constqp -b:v 1M -maxrate:v 10M -rc-lookahead 40 -no-scenecut 1 -init_qpP 23 -init_qpB 25 -init_qpI 21 -weighted_pred 1 "$Vx265"
                t1 = threading.Thread(target=convert, args=(source, '-map 0:v:0 -c:v hevc_nvenc -preset hq -profile:v main10 -tier high -rc:v constqp -b:v 1M -maxrate:v 10M -init_qpP 23 -init_qpB 25 -init_qpI 21', Vx265), daemon = True)
                waiting = True
                t1.start()
                while waiting:
                  tempt = f"{duration(time.time(),'Time Elapsed')}" #Duration
                  print(f'\r{tempt}', end='')
                  time.sleep(1)

                print ("\b"*(len(temp) + len(tempt)),end='')
                temp = f"video h.265 converted\n"
                titVh = len(temp) #for delete the title video h.265 converted
                print (temp,end='')

                temp = f"Converting the video using copy\n"
                print (temp,end='')
                
                #!ffmpeg -hide_banner -loglevel panic -y -i "$source" -map 0:v:0 -c:v copy -map_chapters 0 -map_metadata 0 "$Vcopy"
                t1 = threading.Thread(target=convert, args=(source, '-map 0:v:0 -c:v copy -map_chapters 0 -map_metadata 0', Vcopy), daemon = True) 
                waiting = True
                t1.start()
                while waiting:
                  tempt = f"{duration(time.time(),'Time Elapsed')}" #Duration
                  print(f'\r{tempt}', end='')
                  time.sleep(1)

                print ("\b"*(len(temp) + len(tempt)),end='')
                temp = f"video copy converted\n"
                titVc = len(temp) #for delete the title video copy converted
                print (temp,end='')

                #convert the video using copy
                temp = f"Converting the audio\n"
                print (temp,end='')
                
                #!ffmpeg -hide_banner -loglevel panic -y -i "$source" -map 0:a -c:a aac -q:a 1 -map_chapters 0 -map_metadata 0 "$Aud"
                t1 = threading.Thread(target=convert, args=(source, '-map 0:a -c:a aac -q:a 1 -map_chapters 0 -map_metadata 0', Aud), daemon = True) 
                waiting = True
                t1.start()
                while waiting:
                  tempt = f"{duration(time.time(),'Time Elapsed')}" #Duration
                  print(f'\r{tempt}', end='')
                  time.sleep(1)
                
                print ("\b"*(len(temp) + len(tempt)),end='')
                temp = f"audio converted\n"
                titA = len(temp) #for delete the audio converted
                print (temp,end='')
            
            
                temp = f"Comparing video size :\n"
                tempt = f"{duration(time.time(),'Time Elapsed')}"
                print (temp + tempt,end='')
                titC = 0
                try :
                    x = os.stat(f"Vcopy.{OUT_FORMAT}").st_size
                    y = os.stat(f"Vx265.{OUT_FORMAT}").st_size
                    if x < 100 or y < 100:
                        fail.append(source)
                        flag = False
                except:
                    fail.append(source)
                    flag = False
                print ("\b"*(len(temp) + len(tempt)),end='')
                #compare the video
                if flag:
                    if x>y:
                        temp = f"Combine Video (h.265) w/ audio\n"
                        tempt = f"{duration(time.time(),'Time Elapsed')}"
                        print (temp + tempt,end='')
                        !ffmpeg -hide_banner -loglevel panic -y -i "$Vx265" -i "$Aud" -map 1:a -map 0:v:0 -c:v copy -c:a copy -map_chapters 0 -map_metadata 0 "$output"
                    else:
                        reenc.append(source)
                        temp = f"Combine Video (copy) w/ audio"
                        tempt = f"\n{duration(time.time(),'Time Elapsed')}"
                        print (temp + tempt,end='')
                        !ffmpeg -hide_banner -loglevel panic -y -i "$Vcopy" -i "$Aud" -map 1:a -map 0:v:0 -c:v copy -c:a copy -map_chapters 0 -map_metadata 0 "$output"
                
                titC = len(temp)
                print ("\b"*len(tempt),end='')
            
                #output
                print ("\b"*(titVh+titVc+titA+titC),end='') #delete the convertion status
                try:
                    os.remove(Vx265)
                    os.remove(Vcopy)
                    os.remove(Aud)
                except:
                    pass

            else:
                #!ffmpeg -i "$source" -y -map 0:v:0 -c:v libx265 -x265-params lossless -map 0:a -c:a aac -q:a 1 -map_chapters 0 -map_metadata 0 "$output"
                t1 = threading.Thread(target=convert, args=(source, '-y -map 0:v:0 -c:v libx265 -x265-params lossless -map 0:a -c:a aac -q:a 1 -map_chapters 0 -map_metadata 0', output)) 
                while t1.isAlive():
                  tempt = f"{duration(time.time(),'Time Elapsed')}" #Duration
                  print(f'\r{tempt}', end='')
                  time.sleep(1)
                t1.join()
        try :
            x = os.stat(output).st_size

            if x > 0:
                if DELETE_SC:
                    os.remove(source)
            else:
                if os.path.exists(output):
                    os.remove(output)
                    fail.append(source)
        except:
            if os.path.exists(output):
                os.remove(output)
                fail.append(source)

        #if CLEAN_OUTPUT:    
        #    clear_output()
        print ("\b"*title,end='')

for f in reenc:
    out = f.replace(PATH_ORI+'/', "")
    print("%s is using copy to keep size small" % (out))
for f in fail:
    out = f.replace(PATH_ORI+'/', "")
    print("%s failed to be converted" % (out))

#disable due bug
from google.colab import output
output.eval_js('new Audio("https://drive.google.com/uc?export=download&id=1h-KhKaO-FNMsRKKYtiZ-AqNy8tr5mHcX").play()')
print("done")
raise SystemExit()

In [ ]:
#@title Converting a single file
#@markdown Sometime error coz video format.

import os
from IPython.display import clear_output

SOURCE = ""#@param {type:"string"}
COMMAND = "-map 0:v:0 -c:v hevc_nvenc -preset hq -profile:v main10 -tier high -rc:v constqp -b:v 1M -maxrate:v 10M -rc-lookahead 40 -no-scenecut 1 -init_qpP 23 -init_qpB 25 -init_qpI 21 -weighted_pred 1 -map 0:a -c:a aac -q:a 1 -map_chapters 0 -map_metadata 0"#@param {type:"string"}

#@markdown Video Format
IN_FORMAT = SOURCE[-3:]
OUT_FORMAT = "mp4" #@param {type:"string"}
#@markdown Delete the SOURCE after convertion?
DELETE_SC = True #@param {type:"boolean"}
GPU = "-hwaccel cuvid" #@param ["-hwaccel cuvid", "-hwaccel auto", ""]

files = []
fail = []
if IN_FORMAT == OUT_FORMAT:
    output = SOURCE.replace("." + IN_FORMAT, "-1." + OUT_FORMAT)
    DELETE_SC = False
else:
    output = SOURCE.replace("." + IN_FORMAT, "." + OUT_FORMAT)
!ffmpeg $GPU -i "$SOURCE" $COMMAND  "$output"

try :
    x = os.stat(output).st_size
    y = os.stat(SOURCE).st_size

    if x > (y + 5242880):
        os.remove(output)
        clear_output
        print("output file too big rather than using copy")
        !ffmpeg -i "$SOURCE" -map 0:a -map 0:v -c:v copy -c:a aac -q:a 1 "$output"
    elif x > 0:
        if DELETE_SC:
        os.remove(SOURCE)
    else:
        if os.path.exists(output):
        os.remove(output)
        fail.append(SOURCE)
except:
    if os.path.exists(output):
        os.remove(output)
    print("failed to see file size")

for f in fail:
    out = f.replace(path+'/', "")
    print("%s failed to be converted" % (out))
print("done")
from google.colab import output
output.eval_js('new Audio("https://drive.google.com/uc?export=download&id=1h-KhKaO-FNMsRKKYtiZ-AqNy8tr5mHcX").play()')

### Tweek the video
Restore the files from the trash first

In [ ]:
#@title Reduce audio bitrate(subfolder)
#@markdown Dont overkill the audio
#@markdown using ffmpeg aac -q:a 1

import os
from IPython.display import clear_output
path = ""#@param {type:"string"}
files = []

fol = path + "/mp4"
if not os.path.exists(fol):
    os.makedirs(fol)
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.mp4' in file:
            files.append(os.path.join(r, file))

for f in files:
    source = f
    output = path +'/mp4'+f.replace(path, "")
    !ffmpeg -i "$source" -map 0:a -map 0:v -c:v copy -c:a aac -q:a 1 "$output"
    clear_output()
print("done")
from google.colab import output
output.eval_js('new Audio("https://drive.google.com/uc?export=download&id=1h-KhKaO-FNMsRKKYtiZ-AqNy8tr5mHcX").play()')